In [11]:
from IPython.utils import io
import pandas as pd
import os
from pathlib import Path
from matplotlib import pyplot as plt
from scipy.signal import find_peaks
import numpy as np
import seaborn as sns
import time
import joblib
from os.path import exists
import shutil
import sys
import time

from sklearn.model_selection import train_test_split

#From my EEG package 
import run_expts
import constants

#Let me see as many results as I want to see
pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)

In [ ]:
classifier_list  = [{'C_I_F': CanonicalIntervalForest()}, {'Dr_C_I_F': DrCIF()}]   

In [ ]:
# expt_1_N3_C_I_F_int_df.h5
# expt_1_N3_Dr_C_I_F_raw_clfs_results.h5

In [3]:
data_types = ['Wake' , 'N1' , 'N2' , 'N3' , 'REM']

expt_num = [1,2,3,4]

clf_list = ['Rocket', 'C_I_F' , 'Dr_C_I_F']

#Results dictionary structure : {model : {expt_type : {folder : results_df }}}
results_dict = {}
for clf in clf_list:
    results_dict[clf] = {1 : {} , 2 : {} , 3: {} , 4 : {}}

files_not_found = []

for num in [1,2,3,4]:
    for datatype in data_types:
        for clf in clf_list:
            try:
                if clf == 'Rocket':
                    save_path = 'Results/Results_3/' + 'expt_' + str(num) + '_' + datatype + '_raw_clfs_results.h5'
                else:
                    save_path = 'Results/Results_3/' +  'expt_' + str(num) + '_' + datatype + '_' + clf + '_raw_clfs_results.h5'
                    
#                 file_name = 'expt_' + str(num) + '_' + datatype + '_raw_clfs_results.h5'
                results_df = pd.read_hdf(save_path , key='df', mode='r')
            except FileNotFoundError:
                files_not_found.append(save_path)
                continue
                
            res_df = run_expts.generate_subset_acc_std(results_df, return_df = True)           
            results_dict[clf][num][datatype] = res_df

In [6]:
for clf in ['Rocket', 'C_I_F' , 'Dr_C_I_F']:
    for num in [1,2]:
        print('Expt Type: ' + str(num))
        not_displayed = []
        for key in list(results_dict[clf][num].keys() ):
            df = results_dict[clf][num][key].copy()
            cols_to_drop = [col for col in df.columns if df[col].mean() < 0.7]
            df = df.drop(cols_to_drop, axis = 1 )
            if len(df.columns) < 2:
                not_displayed.append(key)
                continue
            print(key)
            display(df)
        print(not_displayed)

Expt Type: 1
['Wake', 'N1', 'N2', 'N3', 'REM']
Expt Type: 2
Wake


,random_state,Right Parietal_ROCKET_acc
0,1,0.628571
1,2,0.777778


['N1', 'N2', 'N3', 'REM']
Expt Type: 1
['Wake', 'N1', 'N2', 'N3', 'REM']
Expt Type: 2
Wake


,random_state,Right Frontal_C_I_F_acc
0,1,0.739286
1,2,0.755556


['N1', 'N2', 'N3', 'REM']
Expt Type: 1
['N1', 'N2', 'N3', 'REM']
Expt Type: 2
['N1', 'N2', 'N3', 'REM']


## Put the data in a dataframe structure that can easily be plotted

In [16]:
#Defining the dataframe dictionary to be filled up

expt_info = [ '1. PD vs HC', '2. (PD , PD+RBD) vs HC', '3. (PD + PD&RBD) vs HC vs RBD  multiclass', 
             '4. PD vs PD+RBD vs RBD vs HC  multiclass' ]

df_cols = ['Expt_type num', 'Folder', 'Classifier Name', 'Classifier', 'EEG Type',
       'Brain Region', 'Accuracy', 'Accuracy_std', 'TS processing']

df_dict = {}
for col in df_cols:
    df_dict[col] = []

regions = constants.regions

In [17]:
proc = 'shorter_expts'
for clf in clf_list:
    #Go through EEG types
    for eeg_type in ['Wake' , 'N1' , 'N2' , 'N3' , 'REM']:
               
        for expt_num in [1,2,3,4]:
            #Fill up the results_dict with the appropriate values
            #Grab accuracy and std performance for each region
            try:
                mean_df = pd.DataFrame(results_dict[clf][expt_num][eeg_type_res].mean(axis = 0) ).T
            
            except KeyError:
                continue
            for region in regions:
                #For each region take mean accuracy and std
                region_cols = [col for col in mean_df.columns if region in col]
                acc_col = [col for col in region_cols if 'acc' in col]
                std_col = [col for col in region_cols if 'std' in col]
                region_acc = mean_df[acc_col].values[0][0]
                region_acc_std = mean_df[std_col].values[0][0]
                
                #Now fill up the dataframe
                df_dict['Expt_type num'].append(expt_num)
                df_dict['Folder'].append(eeg_type_res)
                df_dict['Classifier Name'].append(clf)
                df_dict['Classifier'].append(clf)
                df_dict['EEG Type'].append(eeg_type)
                df_dict['Brain Region'].append(region)
                df_dict['Accuracy'].append(region_acc)
                df_dict['Accuracy_std'].append(region_acc_std)
                df_dict['TS processing'].append(proc)

results_df = pd.DataFrame(df_dict)

## This portion of the notebook plots accuracy by region (to view trends)

In [18]:
results_df_sorted = results_df.sort_values(by = ['Accuracy'], ascending = False)
results_df_sorted.head(12)

,Expt_type num,Folder,Classifier Name,Classifier,EEG Type,Brain Region,Accuracy,Accuracy_std,TS processing
276,2,Wake,C_I_F,C_I_F,Wake,Right Frontal,0.747421,0.113890,shorter_expts
354,2,Wake,C_I_F,C_I_F,N3,Right Frontal,0.747421,0.113890,shorter_expts
328,2,Wake,C_I_F,C_I_F,N2,Right Frontal,0.747421,0.113890,shorter_expts
302,2,Wake,C_I_F,C_I_F,N1,Right Frontal,0.747421,0.113890,shorter_expts
380,2,Wake,C_I_F,C_I_F,REM,Right Frontal,0.747421,0.113890,shorter_expts
128,2,Wake,Rocket,Rocket,N2,Right Parietal,0.703175,0.099572,shorter_expts
232,2,Wake,Rocket,Rocket,REM,Right Parietal,0.703175,0.099572,shorter_expts
76,2,Wake,Rocket,Rocket,N1,Right Parietal,0.703175,0.099572,shorter_expts
180,2,Wake,Rocket,Rocket,N3,Right Parietal,0.703175,0.099572,shorter_expts
24,2,Wake,Rocket,Rocket,Wake,Right Parietal,0.703175,0.099572,shorter_expts
